# Youtube URL Linked to News

In [2]:
import psycopg2
import pandas as pd
import config
from matplotlib import pylab as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML
import re
from urllib.parse import parse_qs,urlparse

color_palette = sns.color_palette(palette='muted', n_colors=None, desat=.75)
sns.set(context='notebook', palette=color_palette, style='whitegrid', font='sans-serif', font_scale=1.5, color_codes=False, rc=None)
pd.set_option('display.max_colwidth', -1)
table_styles = [{'selector': 'td',
                 'props': [('min-width', '100px'), ('text-align', 'center')]},
                {'selector': 'tr',
                 'props': [('border-bottom', '1px dotted black')]},
                {'selector': 'th',
                 'props': [('text-align', 'center')]}
               ]

%matplotlib inline

directory = "url_top_lists/"
stream = "comparison"

In [3]:
conn = None
try:
    # read connection parameters
    paramsS17 = config.cfgAzureS17()
    paramsS03 = config.cfgAzureS03()

    paramsF17 = config.cfgAzureF17()
    paramsF03 = config.cfgAzureF03()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    connS17 = psycopg2.connect(**paramsS17)
    connS03 = psycopg2.connect(**paramsS03)
    
    connF17 = psycopg2.connect(**paramsF17)
    connF03 = psycopg2.connect(**paramsF03)

    # create a cursor
    curS17 = connS17.cursor()
    curS03 = connS03.cursor()
    
    curF17 = connF17.cursor()
    curF03 = connF03.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    
    curS17.execute('SELECT version()')
    curS03.execute('SELECT version()')
    curF17.execute('SELECT version()')
    curF03.execute('SELECT version()')
    
    # display the PostgreSQL database server version
    db_version_curS17 = curS17.fetchone()
    db_version_curS03 = curS03.fetchone()
    db_version_curF17 = curF17.fetchone()
    db_version_curF03 = curF03.fetchone()
    
    print(db_version_curS17)
    print(db_version_curS03)
    print(db_version_curF17)
    print(db_version_curF03)

    # close the communication with the PostgreSQL
    curS17.close()
    curS03.close()
    curF17.close()
    curF03.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.9, compiled by Visual C++ build 1800, 64-bit',)


## Util Methods

In [4]:
def compareRows(row, df_to_compare, column_name):
    comparison_row = df_to_compare.loc[df_to_compare[column_name] == row[column_name]]
    if comparison_row.empty:
        comparison = " - "
    else:
        percentage_dif = row['percentage'] - comparison_row['percentage'].values[0]
        difference = "(%s. / %.3f%% / %.3f%%)" % (comparison_row['rank'].values[0], comparison_row['percentage'].values[0], percentage_dif)
        if comparison_row['rank'].values[0] == row['rank']:
            comparison = " = <br>" + difference
        else:
            if comparison_row['rank'].values[0] > row['rank']:
                comparison = " v <br>" + difference
            else:
                comparison = " ^ <br>" + difference
    return comparison

def getOpacity(val):
    value = abs(float(re.findall(r"[-+]?\d*\.\d+|\d+", val.split("/")[2])[0]))
    if value < 0.005:
        return 1
    if value < 0.01:
        return 0.95
    if value < 0.05:
        return 0.8
    if value < 0.1:
        return 0.7
    if value < 0.5:
        return 0.6
    if value < 1:
        return 0.5
    if value < 10:
        return 0.3
    if value < 40:
        return 0.2
    if value < 80:
        return 0.1
    if value < 100:
        return 0.05

def colorComparisonField(val):
    
    if isinstance(val, str):
        if ' ^ ' in val or ' v ' in val:
            return 'background-color: rgba(246, 185, 59, %s)' %getOpacity(val)
        if ' = ' in val:
            return 'background-color: rgba(184, 233, 148, %s)' %getOpacity(val)
        if ' - ' in val and len(val) == 3:
            return 'background-color: #e55039' 
    return ''

def generateRankingDataframe(series, attribute_name):
    size = series.sum()
    rank = []
    parameter = []
    count = []
    percentage = []

    i = 1
    for index, value in series.iteritems():
        rank.append(i)
        parameter.append(index)
        count.append(value)
        percentage.append((value/size)*100)
        i += 1

    data = {'rank': rank, attribute_name: parameter, 'value': count, 'percentage': percentage}
    return pd.DataFrame(data=data)

def generateComparisonDataframes(df1, df2, column_name, size):
    compare_list = []
    for index, row in df1.iterrows():
        if row['rank'] <= size:
            compare_list.append(compareRows(row, df2, column_name))

    data = {'rank': df1['rank'][:size], column_name: df1[column_name][:size], 'value': df1['value'][:size], 'percentage': df1['percentage'][:size],
            'difference (rank / percentage / diff)': compare_list}
    
    df1_compared = pd.DataFrame(data=data)
    df1_compared.set_index(keys='rank', inplace=True)
    
    compare_list = []
    for index, row in df2.iterrows():
        if row['rank'] <= size:
            compare_list.append(compareRows(row, df1, column_name))

    data = {'rank': df2['rank'][:size], column_name: df2[column_name][:size], 'value': df2['value'][:size], 'percentage': df2['percentage'][:size],
            'difference (rank / percentage / diff)': compare_list}
    
    df2_compared = pd.DataFrame(data=data)
    df2_compared.set_index(keys='rank', inplace=True)
    
    return df1_compared, df2_compared

def getPrettyComparisonDataframe(df, title):
    s = df.style.applymap(colorComparisonField)
    s.set_caption(title)
    s.set_table_styles(table_styles)
    return s

## Query URLs - Filtered

In [5]:
urlsF03 = pd.read_sql_query("SELECT * FROM tweets_urls;", connF03 )
print("# of URLs filtered 03: %s" %len(urlsF03))

urlsF17 = pd.read_sql_query("SELECT * FROM tweets_urls;", connF17 )
print("# of URLs filtered 17: %s" %len(urlsF17))

urlsF = urlsF17.append(urlsF03)
print("# of URLs filtered: %s" %len(urlsF))

urlsF.head()

# of URLs filtered 03: 2411523
# of URLs filtered 17: 2107279
# of URLs filtered: 4518802


tweet_id  \
0  988437985363406853   
1  988437986797916160   
2  988437989712957440   
3  988437989704568832   
4  988437990732128256   

                                                                                                                              short_url  \
0  https://www.facebook.com/nadjashah/posts/10215947002527277                                                                             
1  https://www.facebook.com/StageSchoolHamburg/posts/1934173349947359                                                                     
2  http://www.faz.net/aktuell/rhein-main/frankfurt/frankfurter-gutleutviertel-landgericht-erlaubt-drohende-zwangsraeumung-15556433.html   
3  http://vera-lengsfeld.de/2018/04/22/buergerprotest-ueberall-wie-lange-wird-das-noch-verschwiegen/                                      
4  https://ift.tt/2Jm4wnp                                                                                                                 

                                                                                                                                                                                                                                                                                  resolved_url  \
0  https://www.facebook.com/nadjashah/posts/10215947002527277                                                                                                                                                                                                                                    
1  https://www.facebook.com/StageSchoolHamburg/posts/1934173349947359                                                                                                                                                                                                                            
2  http://www.faz.net/aktuell/rhein-main/frankfurt/frankfurter-gutleutviertel-landgericht-erlaubt-drohende-zwangsraeumung-15556433.html                                                                                                                                                          
3  http://vera-lengsfeld.de/2018/04/22/buergerprotest-ueberall-wie-lange-wird-das-noch-verschwiegen/                                                                                                                                                                                             
4  https://news.google.com/?sa=t&fd=R&ct2=de&usg=AFQjCNFSz3Yf_fDFIytLtnNe8JNGM2BPZg&clid=c3a7d30bb8a4878e06b80cf16b898331&ei=O_rdWqDIHNCT3QHCyJDQAg&url=https://www.waz.de/kultur/fuer-silke-j-raebiger-ist-es-das-letzte-frauen-filmfestival-id214100661.html&taa=1&hl=en-US&gl=US&ceid=US:en   

   response_code                     domain   top_level_domain  \
0  200.0          https://www.facebook.com/  facebook.com        
1  200.0          https://www.facebook.com/  facebook.com        
2  200.0          http://www.faz.net/        faz.net             
3  200.0          http://vera-lengsfeld.de/  vera-lengsfeld.de   
4  200.0          https://news.google.com/   google.com          

   is_twitter_url is_media  is_processed  failed  
0  False           False    True          False   
1  False           False    True          False   
2  False           False    True          False   
3  False           False    True          False   
4  False           False    True          False

## Query Hashtags - Filtered

In [6]:
hashtagsF03 = pd.read_sql_query("SELECT * FROM tweets_hashtags;", connF03 )
print("# of hashtags filtered 03: %s" %len(hashtagsF03))

hashtagsF17 = pd.read_sql_query("SELECT * FROM tweets_hashtags;", connF17 )
print("# of hashtags filtered 17: %s" %len(hashtagsF17))

hashtagsF = hashtagsF17.append(hashtagsF03)
print("# of URLs filtered: %s" %len(hashtagsF))

hashtagsF.head()

# of hashtags filtered 03: 4377831
# of hashtags filtered 17: 3780197
# of URLs filtered: 8158028


tweet_id        hashtag
0  988175612497063936  Mercedes     
1  988175612497063936  Perkasie     
2  988175612497063936  carsforsale  
3  988175612945854465  reiseagentur 
4  988175612945854465  cancelflights

## Youtube Links with Hashtags

In [ ]:
youtube_urls_f = urlsF[urlsF['top_level_domain'] == 'youtube.com']
print("# of URLs in Dataframe: %s" %len(youtube_urls_f))
i = 0

youtube_value_dict_f = {}
for url, count in youtube_urls_f['resolved_url'].value_counts().iteritems():
    i += 1
    if i % 100000 == 0:
        print("worked ... ",i)
    
    video_tweets_url = youtube_urls_f[youtube_urls_f['resolved_url'] == url]
    hashtags = hashtagsF[hashtagsF['tweet_id'].isin(video_tweets_url['tweet_id'])]

    hashtags_ranking = hashtags['hashtag'].value_counts()
    hashtag_with_values = {}
    for tag, value in hashtags_ranking.iteritems():
        hashtag_with_values[tag] = value
    
    youtube_value_dict_f[url] = hashtag_with_values


# of URLs in Dataframe: 439198


In [89]:
len(youtube_value_dict_f.keys())

5175

## Most Popular official news publishers

In [8]:
top_publisher = ['welt.de', 'spiegel.de', 'focus.de', 'bild.de', 'faz.net', 
                 'sueddeutsche.de', 'zeit.de', 'tagesschau.de', 'presseportal.de', 'tagesspiegel.de',
                 'heise.de', 'n-tv.de', 'epochtimes.de', 'handelsblatt.com', 'derstandard.at',
                 'philosophia-perennis.com', 'journalistenwatch.com', 'tichyseinblick.de', 'taz.de', 'rp-online.de']

## Most Shared Articles with Hashtags

In [10]:
urls_by_publisher_f = urlsF[urlsF['top_level_domain'].isin(top_publisher)]

In [111]:
article_x_yt_matches = {}

for url, count in urls_by_publisher_f['resolved_url'].value_counts().iteritems():
    article_tweets_url = urls_by_publisher_f[urls_by_publisher_f['resolved_url'] == url]
    hashtags = hashtagsF[hashtagsF['tweet_id'].isin(article_tweets_url['tweet_id'])]

    hashtags_ranking = hashtags['hashtag'].value_counts()
    
    matched_urls = []
    matched_values = []
    
    for yt_url, yt_tag_value_dict in youtube_value_dict_f.items():
        yt_url_rating = {}
        matches = 0

        for tag, value in hashtags_ranking.iteritems():
            if tag in yt_tag_value_dict:
                yt_tag_value = yt_tag_value_dict[tag]
                
                if value <= yt_tag_value:
                    matches += value
                else:
                    matches += yt_tag_value

        if matches > 0:
            matched_urls.append(yt_url)
            matched_values.append(matches)
        
    article_x_yt_matches[url] = pd.Series(data=matched_values, index=matched_urls)
            

https://www.tagesschau.de/
http://www.spiegel.de/spiegel/unispiegel/fh-dortmund-bietet-neuen-studiengang-fluechtlingshilfe-an-a-1196053.html
http://www.spiegel.de/politik/deutschland/fluechtlinge-eu-fluechtlingsplaene-alarmieren-bundesregierung-a-1187500.html
https://www.welt.de/vermischtes/article732888/Wenn-der-Cousin-mit-der-Cousine-schlaeft.html
https://www.welt.de/regionales/nrw/article175845813/An-der-Uni-Bielefeld-kann-man-nun-masturbieren-lernen.html
http://www.spiegel.de/politik/deutschland/bayern-in-jeder-behoerde-muss-kuenftig-ein-kreuz-haengen-a-1204523.html
https://www.welt.de/politik/article172675144/Berlin-Fluechtling-soll-4-jaehrige-Tochter-seiner-Pflegefamilie-missbraucht-haben.html
http://www.tagesschau.de/ausland/syrien-truppen-afrin-101.html
https://www.focus.de/politik/deutschland/lage-zu-bedrohlich-demonstration-gegen-antisemitismus-in-berlin-nach-15-minuten-abgebrochen_id_8829384.html
https://philosophia-perennis.com/2018/01/15/karlsruhe-koma/
https://projekte.su

https://www.welt.de/wirtschaft/article172455396/Migration-Auslaender-stellen-ein-Viertel-aller-Arbeitslosen.html
https://www.welt.de/politik/deutschland/article172610868/Roman-Reusch-AfD-Kandidat-fuer-Geheimdienst-Kontrollgremium-im-Bundestag-faellt-durch.html
https://philosophia-perennis.com/2018/04/26/beatrix-von-storch-der-hass-gegen-israel-wird-vom-system-merkel-mitfinanziert/
http://www.spiegel.de/politik/deutschland/spd-und-die-grosse-koalition-der-tod-ist-gar-nicht-so-schlimm-kolumne-a-1187932.html
https://www.n-tv.de/politik/Fluechtlinge-entlasten-Krankenversicherung-article20234356.html
https://www.bild.de/politik/inland/griechenland-krise/und-wieder-sollen-uns-die-griechen-milliarden-kosten-55500322.bild.html
https://www.focus.de/regional/koeln/koeln-fluechtlings-unterkunft-weiterer-fall-bei-koelner-cdu_id_8846356.html?rnd=a2482670da0838f5582f4f19badcbfc3
https://www.welt.de/politik/deutschland/article175909552/Vorschlag-des-Gruenen-Chefs-Fluechtlinge-die-Pfleger-werden-solle

https://www.journalistenwatch.com/2018/01/18/schwedens-ministerpraesident-ich-bin-bereit-das-militaer-einzusetzen-um-die-gewalt-zu-bekaempfen/
https://www.focus.de/politik/deutschland/ralf-stegner-spd-vize-wirft-csu-vor-sondierungspapier-geaendert-zu-haben_id_8297515.html
http://www.spiegel.de/politik/deutschland/grosse-koalition-ist-die-spd-eine-europapartei-gastbeitrag-a-1188743.html
https://www.tagesspiegel.de/berlin/berlin-mitte-polizeibekannte-fluechtlinge-rauben-radfahrer-im-tiergarten-aus/21201654.html
https://www.tagesspiegel.de/weltspiegel/frankreich-jahrhundertkoch-paul-bocuse-ist-tot/20869594.html
https://www.welt.de/finanzen/immobilien/article172460804/Wohnungsmarkt-Die-Fluechtlingskrise-hat-alles-veraendert.html
http://www.sueddeutsche.de/politik/prantls-blick-ein-gesetz-das-angst-und-schrecken-bringt-1.3955373
http://www.spiegel.de/wirtschaft/soziales/jeder-zweite-scheitert-im-integrationskurs-am-deutschtest-a-1205397.html
https://www.bild.de/news/inland/urteil/auch-sie-k

http://www.epochtimes.de/politik/deutschland/neun-staedte-33-stunden-deutschland-messer-attacken-in-bonn-heidelberg-essen-co-a2326761.html?tweet=1
https://www.tichyseinblick.de/daili-es-sentials/vor-der-groko-weine-nicht-kleine-spd/
https://www.heise.de/newsticker/meldung/Facebook-Fake-News-werden-zukuenftig-kleiner-dargestellt-4037502.html
https://www.focus.de/politik/ausland/korea-gipfel-nord-und-suedkorea-wollen-getrennte-familien-wieder-zusammenbringen_id_8839496.html
https://www.welt.de/kultur/medien/article172481944/Kika-Protagonist-auf-Facebook-Das-war-als-Scherz-gemeint.html?wtmc=socialmedia.twitter.shared.web
http://www.tagesschau.de/inland/unwort-des-jahres-105.html
https://www.welt.de/politik/ausland/article172676449/Tuerkische-Bodentruppen-marschieren-in-Syrien-ein.html
https://www.welt.de/politik/ausland/plus172496254/Antisemitismus-in-Frankreich-Die-Angst-vor-dem-linken-Judenhass-waechst.html
https://www.zeit.de/kultur/2018-01/oeffentlich-rechtlicher-rundfunk-rundfunkbeit

https://www.n-tv.de/politik/Gericht-verurteilt-Puigdemont-Partei-article20232576.html
https://www.welt.de/politik/deutschland/article175898793/Bundestagsdebatte-Kann-man-Hetze-gegen-Deutsche-bestrafen.html
https://www.bild.de/politik/inland/nachrichtendienste-iran/razzien-bei-iranischen-agenten-54498494.bild.html
https://www.welt.de/politik/deutschland/article172455051/Altersfeststellung-Viele-der-angeblich-Minderjaehrigen-als-volljaehrig-eingestuft.html
http://www.spiegel.de/politik/ausland/eu-will-whistleblower-besser-schuetzen-nach-luxleaks-cambridge-analytica-a-1204244.html
http://www.spiegel.de/gesundheit/diagnose/donald-trump-was-wissen-wir-wirklich-ueber-seine-geistige-gesundheit-a-1188295.html
http://www.spiegel.de/politik/ausland/emmanuel-macron-und-seine-migrationspolitik-humanitaet-oder-haerte-a-1188497.html
http://www.faz.net/aktuell/wirtschaft/eurokrise/griechenland-hat-weitere-milliardenzahlungen-in-aussicht-15563470.html?GEPC=s3
http://www.tagesschau.de/inland/spd-sonder

...

https://www.focus.de/regional/wuerzburg/finanzen-anleihe-aus-dem-jahr-1901-unterm-hammer_id_8327532.html
http://www.faz.net/aktuell/politik/trumps-praesidentschaft/donald-trump-bleibt-dinner-mit-journalisten-fern-15565855.html?GEPC=s3
https://www.journalistenwatch.com/2018/04/28/joerg-meuthen-ueber-den-scharia-mord-an-baby-die-buerger-haben-ein-recht-darauf-die-ganze-wahrheit-zu-erfahren/
https://www.journalistenwatch.com/2018/04/24/hallo-sozialverbaende-wo-bleibt-euer-nazi-vergleich-barley-draengt-auf-aenderung-von-paragraf-219a/
https://derstandard.at/2000072514481/Russischer-Neofaschist-Dugin-kommt-vor-Akademikerball-nach-Wien
https://www.journalistenwatch.com/2018/04/29/brief-eines-einwanderers-an-michael-klonovsky-ist-hier-der-galoppierende-wahnsinn-ausgebrochen/
https://jobs.zeit.de/jobs/keine_angabe_referent_m_w_arbeitswissenschaft_152464.html?partner=DAADGE&wt_zmc=sm.int.zonpmr.zeitde.stellenanzeige.stellenmarkt.feed.Jobs%20Niedersachsen.x&utm_medium=sm&utm_source=zeitde_zonpmr

http://www.spiegel.de/fotostrecke/kim-jong-un-trifft-moon-jae-in-fotostrecke-160338.html
https://www.tagesspiegel.de/wirtschaft/internetverbindungen-weshalb-mobiles-internet-oft-zu-langsam-ist/20858494.html
https://www.presseportal.de/blaulicht/pm/126723/3841953?utm_source=dlvr.it&utm_medium=twitter
https://www.zeit.de/kultur/literatur/2018-01/buchmarkt-lesehund-stille-lektuere
http://www.spiegel.de/wirtschaft/unternehmen/gehaelter-das-sind-die-bestbezahlten-konzernwaechter-a-1204909.html
https://www.bild.de/bild-plus/sport/mehr-sport/handball/tobias-reichmann-im-grossen-interview-54496994,view=conversionToLogin.bild.html?utm_source=dlvr.it&utm_medium=twitter
https://www.zeit.de/politik/ausland/2018-04/syrien-aleppo-diktatur-baschar-al-assad-vertreibung?utm_source=twitter_zonaudev_int&wt_zmc=sm.int.zonaudev.twitter.ref.zeitde.redpost.link.sf&utm_content=zeitde_redpost+_link_sf&utm_campaign=ref&utm_medium=sm
http://www.faz.net/aktuell/wirtschaft/netzkonferenz-dld/carl-bendikt-frey-ueber

https://derstandard.at/2000071335171/Das-traditionelle-Japan-zu-Beginn-des-20-Jahrhunderts
http://www.handelsblatt.com/handball-weitere-nachnominierung-bei-dhb-team-vor-daenemark-spiel/20869250.html
https://www.zeit.de/news/2018-04/23/wochenende-ueberschattet-von-toedlichen-biker-unfaellen-180423-99-05809?wt_zmc=sm.ext.zonaudev.twitter.ref.zeitde.share.link.x
https://www.n-tv.de/mediathek/videos/wirtschaft/Wer-bei-einer-Zinswende-verliert-und-gewinnt-article20405365.html
http://www.faz.net/aktuell/finanzen/finanzmarkt/american-express-vermeldet-quartalsverlust-von-1-2-milliarden-dollar-15406148.html
https://rp-online.de/nrw/staedte/duesseldorf/stadtgespraech/gemeinsam-jeck-im-uerige_aid-17673165
http://www.sueddeutsche.de/sport/handball-fuechse-berlin-holen-spektakulaer-acht-tore-auf-1.3962321
http://www.spiegel.de/sport/fussball/champions-league-as-rom-vs-fc-liverpool-edin-dzeko-der-alleskoenner-a-1204301.html
https://www.zeit.de/digital/internet/2018-01/soziale-netzwerke-facebook-twi

https://www.zeit.de/wissen/gesundheit/2018-04/psychiatrie-gesetz-bayern-psychkhg-stigmatisierung-psychisch-kranke-nationalsozialismus?wt_zmc=sm.ext.zonaudev.twitter.ref.zeitde.share.link.x
https://www.welt.de/politik/ausland/article172552897/Cappuccino-Index-Ist-die-EU-Mitgliedschaft-billiger-als-ein-Kaffee.html
http://www.tagesschau.de/eilmeldung/eilmeldung-3325.html
https://www.zeit.de/2018/16/soziale-netzwerke-facebook-konkurrenz-datenschutz?wt_zmc=sm.ext.zonaudev.twitter.ref.zeitde.share.link.x
https://philosophia-perennis.com/2018/01/16/yasemin-paria/
https://www.presseportal.de/blaulicht/pm/126720/3930257
https://www.bild.de/unterhaltung/tv/dschungelcamp/die-luxus-gegenstaende-der-stars-54504092.bild.html
https://m.bild.de/bild-plus/regional/duesseldorf/osama-bin-laden/so-viel-kassiert-sein-ex-leibwaechter-55486374,view=conversionToLogin,oview=amp.bildMobile.html?__twitter_impression=true
https://www.focus.de/regional/hannover/bundespolizeiinspektion-hannover-brutalen-schlaeger-n

https://www.zeit.de/wissen/umwelt/2018-04/bienensterben-ursachen-pestizide-imker-klimawandel?wt_zmc=sm.ext.zonaudev.twitter.ref.zeitde.share.link.x
http://www.tagesschau.de/inland/asylsuchende-statistik-101.html?utm_medium=twitter&utm_source=dlvr.it
https://www.focus.de/politik/deutschland/juedische-verbaende-besorgt-antisemitismus-auf-dem-schulhof-jede-woche-gibt-es-vorfaelle-selbst-an-kitas-ein-thema_id_8673391.html?rnd=02c30079b124960b5dd0ff0f8b7f9055
https://www.presseportal.de/blaulicht/pm/13248/3840542?utm_source=dlvr.it&utm_medium=twitter
https://www.tagesspiegel.de/berlin/wohnungsnot-in-berlin-riesige-brache-in-marzahn-wird-privatisiert/21201354.html
https://www.bild.de/politik/ausland/italien/putin-fan-will-italien-regieren-54441534.bild.html
https://www.presseportal.de/blaulicht/pm/74167/3843476
https://www.focus.de/regional/thueringen/landespolizeiinspektion-gotha-hoher-sachschaden-durch-gesprengten-briefkasten-zeugen-gesucht_id_8298640.html?rnd=47fda8c6721c479de871037644a98

## Articles x Videos by Hashtag co-occurences

In [127]:
def getHashtagsFromURL(url, urls, hashtags):
    return hashtags[ hashtags['tweet_id'].isin(urls[urls['resolved_url'] == url]['tweet_id'])]

In [156]:
rank=1
for publisher in top_publisher[:1]:
    all_urls_by_publisher = urlsF[urlsF['top_level_domain'] == publisher]
    
    articles_counted_f = all_urls_by_publisher['resolved_url'].value_counts()
    
    print("rank", "publisher", "# shares", "# unique", "% unique", sep="\t")
    
    print(rank, publisher,
          len(all_urls_by_publisher),
          len(articles_counted_f),
          "%.2f%%" %(len(articles_counted_f)/len(all_urls_by_publisher)*100), 
          
          sep='\t')
    rank+=1
    print("", "#", "%", "article", "hashtags", sep="\t")
    
    for article, count in articles_counted_f[:5].iteritems():
        print("",
              count, 
              "%.2f%%" %(count/sum(articles_counted_f)*100), 
              article,
              ", ".join(getHashtagsFromURL(article, urlsF, hashtagsF)['hashtag'].unique()),
              sep='\t')
        
        if article in article_x_yt_matches:
            article_x_yt_matches[article].sort(ascending=False)
            print("", "", "#", "Youtube URL", "Hashtags", sep="\t")
            for matched_yt_url, count in article_x_yt_matches[article][:5].iteritems():
                print("",
                      "",
                      count,
                      matched_yt_url,
                      ", ".join(getHashtagsFromURL(matched_yt_url, urlsS, hashtagsF)['hashtag'].unique()),
                      sep="\t")
        else:
            print("", "", "", "", "", sep="\t")

rank	publisher	#	# unique	% unique
1	welt.de	1222	739	60.47%
	#	%	article	hashtags
	12	0.98%	https://www.welt.de/vermischtes/article732888/Wenn-der-Cousin-mit-der-Cousine-schlaeft.html	FluechtlingsIndustrie, Sozialverbände, Altparteien, Staatsmedien, FakeNews, Welt, Inzucht, taz, AfD, Anfrage, Fakten, Doppelmoral
		#	Youtube URL	Hashtags
		3	https://www.youtube.com/channel/UC_dZp8bZipnjntBGLVHm6rw/about?sub_confirmation=1	Bremen, AfD, Bundestag, Diesel, G7, Russland, Israel, UNWRA, BAMF, Volksverhetzung, Otten
		2	https://www.youtube.com/channel/UCNz-EGSd5BXEoTcLcYCGsig/	AfD, Wildberg, Bundestag, AfDwirkt
		2	https://www.youtube.com/watch?v=eKrc5ripDbU&feature=youtu.be	Weidel, AfD
		2	https://www.youtube.com/watch?v=JXZFCQXApEY	Hampel, Wendezeit, Syrien, AfD, Bundestag, Landtag, Syrienreise, AfDwirkt, Retweet
		2	https://www.youtube.com/watch?v=r6hvOcIlS4M	AfD, Wildberg, Bundestag, AfDwirkt
	11	0.90%	https://www.welt.de/regionales/nrw/article175845813/An-der-Uni-Bielefeld-kann-man-nun-

## Youtube TOP URLs compared to articles

In [17]:
def getHashtagsFromURL(url, urls, hashtags):
    return hashtags[ hashtags['tweet_id'].isin(urls[urls['resolved_url'] == url]['tweet_id'])]

def compareHashtags(hashtags_a_dict, hashtags_b_dict):
    matches = 0
    for hashtag_a in hashtags_a_dict.keys():
        if hashtag_a in hashtags_b_dict:
            if hashtags_a_dict[hashtag_a] <= hashtags_b_dict[hashtag_a]:
                matches += hashtags_a_dict[hashtag_a]
            else:
                matches += hashtags_b_dict[hashtag_a]
    if matches > 0:
        print(matches)
        print(hashtags_a_dict, hashtags_b_dict)

def common_member(a, b):
    
    a_set = set(a)
    b_set = set(b)
    
    if (a_set & b_set):
        print(a_set, b_set)
        return True
    else:
        return False

In [8]:
top_publisher = ['welt.de', 'spiegel.de', 'focus.de', 'bild.de', 'faz.net', 
                 'sueddeutsche.de', 'zeit.de', 'tagesschau.de', 'presseportal.de', 'tagesspiegel.de',
                 'heise.de', 'n-tv.de', 'epochtimes.de', 'handelsblatt.com', 'derstandard.at',
                 'philosophia-perennis.com', 'journalistenwatch.com', 'tichyseinblick.de', 'taz.de', 'rp-online.de']

urls_by_publisher = urlsF[urlsF['top_level_domain'].isin(top_publisher)]
print(len(urls_by_publisher))

418306


In [ ]:
hashtags_by_article_url = {}
distinct_urls = urls_by_publisher['resolved_url'].unique()
print(len(distinct_urls))
for article in distinct_urls:
    hashtags = getHashtagsFromURL(article, urlsF, hashtagsF)
    
    if len(hashtags>0):
        hashtags_by_article_url[article] = hashtags['hashtag'].value_counts().to_dict()
    


115643


In [26]:
hashtags_by_article_url

{'http://www.faz.net/aktuell/politik/der-moderne-krieg-wir-muessen-vorbereitet-sein-15548472.html?GEPC=s3': {'Europa': 1,
  'Krisenvorbereitung': 1,
  'Krisenvorsorge': 1,
  'Schweden': 1},
 'http://www.faz.net/aktuell/rhein-main/frankfurt/brillenbaeren-nachwuchs-erkundet-aussenanlage-des-frankfurter-zoos-15556243.html': {},
 'http://www.faz.net/aktuell/rhein-main/frankfurt/frankfurter-gutleutviertel-landgericht-erlaubt-drohende-zwangsraeumung-15556433.html': {},
 'http://www.faz.net/aktuell/technik-motor/technik/welche-kaffeemaschine-kaufen-zwei-kaffeemaschinen-im-test-15553950.html?GEPC=s3': {},
 'http://www.faz.net/aktuell/wirtschaft/ministerin-will-peta-im-zweifel-gemeinnuetzigkeit-nehmen-15556150.html': {'NGODebatte': 2,
  'Niedersachsen': 1,
  'PETA': 1,
  'Tierrechtsorganisationen': 1,
  'Tierschutz': 2,
  'ottekinast': 1,
  'politischeKultur': 1},
 'http://www.faz.net/aktuell/wirtschaft/schadensersatz-neuer-streit-zwischen-vw-und-prevent-15556380.html': {'Prevent': 4,
  'VW': 4

In [16]:
distinct_urls

{'https://rp-online.de/nrw/panorama/essen-frau-17-mit-flasche-auf-kopf-geschlagen-mittaeter-festgenommen_aid-20025323',
 'https://www.n-tv.de/sport/fussball/Alkoholverbot-Bloss-keine-Panik-article20405608.html',
 'http://www.epochtimes.de/politik/deutschland/union-lehnt-nachbesserungen-am-sondierungskompromiss-ab-2-a2321417.html',
 'https://m.tagesspiegel.de/politik/oskar-groening-ex-ss-mann-reicht-gnadengesuch-ein/20850610.html?utm_referrer=http%3A%2F%2Fm.facebook.com',
 'https://www.presseportal.de/blaulicht/pm/126309/3843897?utm_source=dlvr.it&utm_medium=twitter',
 'https://www.focus.de/regional/polizei-naila-mercedes-fahrer-verunglueckt-auf-winterglatter-bundesstrasse_id_8331252.html?rnd=757ff1532767c4d91b2390d46ab5643c',
 'https://www.heise.de/mac-and-i/meldung/Bericht-Teile-des-Codes-von-iPhone-Unlock-Tool-GrayKey-entfleucht-4032877.html',
 'https://www.presseportal.de/blaulicht/pm/11562/3929728?utm_source=dlvr.it&utm_medium=twitter',
 'http://www.spiegel.de/politik/ausland/fluec

In [13]:
youtube_urls = urlsF[urlsF['top_level_domain'] == 'youtube.com']

for yt_url, count in youtube_urls['resolved_url'].value_counts()[1:10].iteritems():
    #print(yt_url, count)
    hashtags_yt_url = getHashtagsFromURL(yt_url, urlsF, hashtagsF)
    #print(hashtags_yt_url['hashtag'].value_counts())
    if len(hashtags_yt_url > 0):
        for article_url in urls_by_publisher['resolved_url'][:10].values:
            #print(article_url)
            hashtags_article_url = getHashtagsFromURL(article_url, urlsF, hashtagsF)
            if len(hashtags_article_url) > 0:
                #print(article_url)
                #print(hashtags_article_url['hashtag'].value_counts().to_dict())
                if common_member(hashtags_yt_url['hashtag'].values,
                                 hashtags_article_url['hashtag'].values):
                    print(hashtags_yt_url['hashtag'].values)
                    print(hashtags_article_url['hashtag'].values)
                    #compareHashtags(hashtags_yt_url['hashtag'].value_counts().to_dict(), 
                     #               hashtags_article_url['hashtag'].value_counts().to_dict())
        

{'dankechico', 'funk', 'Trolle', 'noafd', 'gegenhass', 'gesamtsituation', 'Twitter', 'Deutschland3000', 'Trollfabriken', 'ReconquistaGermanica', 'discord', 'ReconquistaInternet', 'Netzwerkanalyse', 'Latte', 'Internet', 'böhmermann', 'NoNazis', 'NoAfD', 'neomagazin', 'Meinungsfreiheit', 'reconquistagermanica', 'sichtempfehlung', 'FakeNews', 'noAfD', 'internet', 'Propaganda', 'OrnanistaGermaninca', 'ZDFneo', 'Trollkacke', 'Zombies', 'Raykdoku', 'reconquistainternet', 'ThePowerOfARDundZDF', 'OnanistaGermanica', 'Muschi', 'Hass', 'Datenanalyse', 'freepepe', 'AfD', 'DankeChico', 'fcknzs', 'Böhmi', 'GegenHassundHetze', 'YouTube'} {'Zensur', 'internet', 'humanrights', 'China'}
['funk' 'ReconquistaInternet' 'DankeChico' 'ThePowerOfARDundZDF'
 'neomagazin' 'ReconquistaInternet' 'ReconquistaInternet' 'dankechico'
 'ReconquistaInternet' 'ReconquistaInternet' 'dankechico'
 'ReconquistaInternet' 'dankechico' 'ReconquistaInternet'
 'ReconquistaInternet' 'ReconquistaInternet' 'ReconquistaInternet'
 '